In [95]:
from __future__ import print_function
from keras.layers import Embedding
from keras.initializers import Constant
import nltk
import io
import numpy as np 
import csv
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, GRU, Dense, Concatenate, Flatten, Average, Activation
from keras.layers.wrappers import Bidirectional
from keras.activations import softmax

In [63]:
line_cnt = 0 
new_line = False
sent_id = 1
all_sents = []
all_tags = []

with open('train.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    
    sent_seq = ""
    tag_seq = ""
    for row in csv_reader:
        if line_cnt != 0:
            
            if sent_id != row[2]:
                new_line = True
                all_sents.append(sent_seq)
                all_tags.append(tag_seq)
            else:
                new_line = False
            
            word_id = row[0]
            doc_id  = row[1]
            sent_id = row[2]
            word    = row[3]
            tag     = row[4]
            
            if new_line == False:
                sent_seq = sent_seq + ' ' + word
                tag_seq = tag_seq + ' ' + tag
            
            else:
                sent_seq = word 
                tag_seq = tag
            
        line_cnt = line_cnt + 1

In [64]:
#len(all_tags)
#len(all_sents)
    
all_tags = [item.replace('B-indications','B') for item in all_tags]
all_tags = [item.replace('I-indications','I') for item in all_tags]

In [65]:
#get max sequences 
def getMaxSeq(List):
        
    max_len = 0
    
    for item in List:
        l = len(item.split())
        if l > max_len:
            max_len = l
    
    return max_len

In [66]:
MAX_LEN = 575 #timesteps

In [67]:
#get vocab of sentences, tags
def getVocabSets(sents_list, tags_list):
    sents_vocab_set = set()
    tags_vocab_set = set()
    common_vocab_set = set()
    
    for item in sents_list:
        item = item.decode('utf8')
        for word in nltk.word_tokenize(item):
            sents_vocab_set.add(word.lower())
        
    for item in tags_list:
        for word in nltk.word_tokenize(item):
            tags_vocab_set.add(word.lower())
        
    common_vocab_set.update(sents_vocab_set)
    common_vocab_set.update(tags_vocab_set)
    
    return (common_vocab_set, sents_vocab_set, tags_vocab_set)

In [68]:
common_vocab, sents_vocab, tags_vocab = getVocabSets(all_sents, all_tags)

In [69]:
"""""
all_sents_enc = []
all_tags_enc = []

#for item in all_sents:
#    print(item)
#    item = item.decode('utf8')
#    enc_sents_list.append(item)
    
for item in all_tags:
    item = item.decode('utf8')
    if len(item.split()) > 575:
        print('hello')

"""""


'""\nall_sents_enc = []\nall_tags_enc = []\n\n#for item in all_sents:\n#    print(item)\n#    item = item.decode(\'utf8\')\n#    enc_sents_list.append(item)\n    \nfor item in all_tags:\n    item = item.decode(\'utf8\')\n    if len(item.split()) > 575:\n        print(\'hello\')\n\n'

In [70]:
#embedding dimension
EMBEDDING_DIM = 100

In [71]:
# function to LOAD Glove Embeddings
def loadGloVeModel(gloVe_file):
    embedding_model = {}
    
    with io.open(gloVe_file, encoding = 'utf8') as f:
        word_embeddings = f.readlines()
        
    for word_embedding_line in word_embeddings:
        word_embedding = word_embedding_line.split()
        word = word_embedding[0]
        embedding = np.array([float(col) for col in word_embedding[1:]])
        embedding_model[word] = embedding
            
    return embedding_model

In [72]:
# embeddings loaded
embedding_model = loadGloVeModel('radglove.800M.100d.txt')

In [73]:
# sents_vocab.intersection(tags_vocab) # {'o'}

In [74]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(common_vocab)

word_to_index = tokenizer.word_index

sents_sequences = tokenizer.texts_to_sequences(all_sents)
tags_sequences = tokenizer.texts_to_sequences(all_tags)

sents_data = pad_sequences(sents_sequences, maxlen = MAX_LEN, padding = 'post')
tags_data = pad_sequences(tags_sequences, maxlen = MAX_LEN, padding = 'post')

In [75]:
# get the embeddings in a matrix form
embedding_matrix = np.zeros((len(common_vocab),EMBEDDING_DIM))

for word,index in word_to_index.items():
    embedding_vector = embedding_model.get(word)
    
    if embedding_vector is not None :
        embedding_matrix[index] = embedding_vector

In [76]:
#decide maximum sequence length later...
MAX_SEQUENCE_LENGTH = MAX_LEN

# check doc.s once
# trainable=False to prevent the weights from being updated during training
embedding_layer = Embedding(len(common_vocab), EMBEDDING_DIM, embeddings_initializer=Constant(embedding_matrix), input_length=MAX_SEQUENCE_LENGTH, trainable=False)

In [77]:
all_sents[999]

'PAP/REG3A favors perineural invasion in pancreatic adenocarcinoma and serves as a prognostic marker .'

In [78]:
index_to_word = {}
for word,index in word_to_index.items():
    index_to_word[index] = word

In [79]:
for i in sents_data[999]:
    if i == 0:
        continue
    print(index_to_word[i], end = " ")

pap reg3a favors perineural invasion in pancreatic adenocarcinoma and serves as a prognostic marker 

In [80]:
sents_data[999]

array([ 4355, 72484, 73822, 78764,  7664,    81,  5380,  4438,   223,
       54726,   305,    29, 11110,  2604,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [81]:
tags_data[999]

array([47, 47, 47, 47, 47, 61, 60, 47, 47, 47, 47, 47, 47, 47,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0

In [82]:
all_tags[999]

'O O O O O B I O O O O O O O'

In [29]:
epochs = 2
batch_size = 10

In [31]:
"""""

nn_input = Input(shape = (MAX_LEN,) , dtype = 'int32')

nn_input = embedding_layer(nn_input)

nn_out, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(100, return_sequences = True, return_state = True))(nn_input)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

#nn_output = Dense(,activation = 'sigmoid')(nn_output)

print(nn_out.shape)

nn_output = softmax(nn_out)


model = Model(inputs = nn_input, outputs = nn_output)

"""""

'""\n\nnn_input = Input(shape = (MAX_LEN,) , dtype = \'int32\')\n\nnn_input = embedding_layer(nn_input)\n\nnn_out, forward_h, forward_c, backward_h, backward_c = Bidirectional(LSTM(100, return_sequences = True, return_state = True))(nn_input)\n\nstate_h = Concatenate()([forward_h, backward_h])\nstate_c = Concatenate()([forward_c, backward_c])\n\n#nn_output = Dense(,activation = \'sigmoid\')(nn_output)\n\nprint(nn_out.shape)\n\nnn_output = softmax(nn_out)\n\n\nmodel = Model(inputs = nn_input, outputs = nn_output)\n\n'

In [157]:
nn_input = Input(shape = (MAX_LEN,) , dtype = 'int32')

nn_embed = embedding_layer(nn_input)

print(nn_embed.shape)

nn_out1 = GRU(575, return_sequences = False)(nn_embed)
nn_out2 = GRU(575, return_sequences = False, go_backwards = True)(nn_embed)

print()

print(nn_out1.shape)
print(nn_out2.shape)

nn_out = Average()([nn_out1, nn_out2])

print()
print(nn_out.shape)

#nn_output = Dense(,activation = 'sigmoid')(nn_output)

#print(nn_out.shape)

#nn_output = softmax(nn_out)
#nn_out = Flatten()(nn_out)
nn_output = Activation('softmax')(nn_out)
print(nn_output.shape)
#print(nn_output.shape)

model = Model(inputs = nn_input, outputs = nn_output)

(?, 575, 100)

(?, 575)
(?, 575)

(?, 575)
(?, 575)


In [158]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy')

In [159]:
model.fit(sents_data, tags_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)

Train on 153025 samples, validate on 38257 samples
Epoch 1/2


KeyboardInterrupt: 

In [126]:
tags_data.shape

(191282, 575)